# Ivy — WGU Course Pain Point and Help-Seeking Analysis

**Purpose**  
This notebook runs the complete Ivy pipeline, from raw Reddit posts to final JSON outputs, in five steps:

1. Preprocessing: Query raw Reddit posts from the SQLite database, extract course codes, score sentiment, and filter for strong negative signals  
2. Taxonomy pass  
3. Cluster detection  
4. Root cause labeling  
5. Peer advice and help-seeking pattern extraction

The output is structured JSON files for each course, ready for staff review and dashboards.

**Input:** `/sqlite/wgu_reddit.db`  
**Intermediate:** `/outputs/preprocessed/{CourseCode}_clean_posts.csv`  
**Output:** `/outputs/final/`  
**Capstone Tracker:** `reddit-pipeline-status-v2.2`

In [ ]:
import pandas as pd
import sqlite3
import json
import os
from datetime import datetime
from openai import OpenAI
from typing import Dict, List, Optional

In [2]:
class IvyAnalyzer:
    def __init__(self, api_key: str, db_path: str = "db/WGU-Reddit.db") -> None:
        """
        Initialize Ivy analyzer with OpenAI client and database connection.
        """
        self.client = OpenAI(api_key=api_key)
        self.db_path = db_path
        self.model = "gpt-4o-mini"

    def get_filtered_posts(self, course_code: Optional[str] = None, sentiment_threshold: float = -0.2) -> pd.DataFrame:
        """
        Preprocessing: Query raw Reddit posts from SQLite database and filter for strong negative sentiment.
        """
        with sqlite3.connect(self.db_path) as conn:
            if course_code:
                query = """
                SELECT p.id, p.title, p.selftext, p.created_utc, p.score, p.num_comments,
                       p.vader_score, pk.course_codes, p.subreddit
                FROM posts p 
                JOIN posts_keyword pk ON p.id = pk.post_id
                WHERE p.vader_score < ? AND pk.course_codes LIKE ?
                ORDER BY p.created_utc DESC
                """
                params = (sentiment_threshold, f"%{course_code}%")
            else:
                query = """
                SELECT p.id, p.title, p.selftext, p.created_utc, p.score, p.num_comments,
                       p.vader_score, pk.course_codes, p.subreddit
                FROM posts p 
                JOIN posts_keyword pk ON p.id = pk.post_id
                WHERE p.vader_score < ?
                ORDER BY p.created_utc DESC
                """
                params = (sentiment_threshold,)

            df = pd.read_sql_query(query, conn, params=params)

        print(f"Retrieved {len(df)} posts for analysis")
        return df

    def taxonomy_pass(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Step 2: Taxonomy pass - Extract pain points, categories, and help-seeking patterns via LLM.
        """
        print(f"Running taxonomy pass on {len(df)} posts")
        results = []

        for _, row in df.iterrows():
            prompt = f"""You are analyzing a WGU student post.\nTitle: {row['title']}\nText: {row['selftext']}\nSentiment: {row['vader_score']}"""

            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You analyze WGU student posts."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3
            )

            # TODO: Replace with actual response parsing
            analysis = {
                'post_id': row['id'],
                'course_codes': row['course_codes'],
                'pain_point': "TODO",  # Replace with parsed result
                'category': "TODO",
                'help_seeking_type': "TODO",
                'severity': "TODO",
                'key_quotes': [],
                'confidence': 0.0,
                'original_title': row['title'],
                'original_text': row['selftext'],
                'sentiment_score': row['vader_score']
            }

            results.append(analysis)

        return pd.DataFrame(results)

    def cluster_detection(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Step 3: Cluster detection - Group similar pain points into clusters.
        Note: We may replace the simple helper later with real clustering.
        """
        print(f"Running cluster detection on {len(df)} items")

        # Decision: Keep inline or factor out helper if clustering expands.
        df['cluster_name'] = df['category'].apply(lambda x: f"{x} Cluster")
        df['cluster_id'] = df['category'].apply(lambda x: f"cluster_{x.lower()}")

        print(f"Clusters found: {df['cluster_name'].nunique()}")
        return df

    def root_cause_labeling(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Step 4: Root cause labeling.
        """
        print("Running root cause labeling")
        df['root_cause'] = df['cluster_name'].apply(lambda x: f"Root cause for {x}")
        df['suggested_actions'] = df['cluster_name'].apply(lambda x: f"Suggested actions for {x}")
        return df

    def peer_advice_and_help_seeking_pattern_extraction(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Step 5: Extract peer advice and help-seeking patterns.
        """
        print("Extracting peer advice/help-seeking patterns")
        df['advice_pattern'] = df['help_seeking_type'].apply(lambda x: f"Advice pattern for {x}")
        df['peer_responses'] = df['post_id'].apply(lambda x: f"Mock peer responses for {x}")
        return df

    def generate_course_analysis(self, course_code: str, df: pd.DataFrame) -> Dict:
        """
        Generate final structured JSON.
        """
        return {
            "course_code": course_code,
            "analysis_date": datetime.now().isoformat(),
            "total_posts_analyzed": len(df),
            "pain_point_summary": {
                "total_pain_points": len(df[df['pain_point'].notna()]),
                "categories": df['category'].value_counts().to_dict(),
                "severity_distribution": df['severity'].value_counts().to_dict(),
                "help_seeking_types": df['help_seeking_type'].value_counts().to_dict()
            },
            "top_clusters": df.groupby('cluster_name').size().head(5).to_dict(),
            "recommended_actions": df['suggested_actions'].unique().tolist()[:5],
            "data_quality": {
                "confidence_avg": df['confidence'].mean(),
                "posts_with_quotes": len(df[df['key_quotes'].apply(len) > 0]),
                "sentiment_range": [df['sentiment_score'].min(), df['sentiment_score'].max()]
            }
        }

    def analyze_course(self, course_code: str) -> Dict[str, any]:
        """
        Run full Ivy pipeline for a single course.
        """
        print(f"=== Analyzing {course_code} ===")
        posts_df = self.get_filtered_posts(course_code)

        if posts_df.empty:
            print(f"No posts found for {course_code}")
            return {}

        taxonomy_df = self.taxonomy_pass(posts_df)
        clustered_df = self.cluster_detection(taxonomy_df)
        labeled_df = self.root_cause_labeling(clustered_df)
        final_df = self.peer_advice_and_help_seeking_pattern_extraction(labeled_df)

        outputs = {
            'final_course_analysis': self.generate_course_analysis(course_code, final_df),
            'processed_posts': final_df.to_dict('records'),
            'summary': {
                'total_posts': len(posts_df),
                'pain_points_identified': len(final_df),
                'clusters_found': final_df['cluster_name'].nunique(),
                'average_confidence': final_df['confidence'].mean()
            }
        }

        self.save_outputs(course_code, outputs)
        print(f"=== Completed analysis for {course_code} ===")
        return outputs

    def save_outputs(self, course_code: str, outputs: Dict) -> None:
        """
        Save JSON outputs.
        """
        os.makedirs("outputs/final", exist_ok=True)

        with open(f"outputs/final/{course_code}_final_course_analysis.json", 'w') as f:
            json.dump(outputs['final_course_analysis'], f, indent=2)

        with open(f"outputs/final/{course_code}_processed_posts.json", 'w') as f:
            json.dump(outputs['processed_posts'], f, indent=2)

        print(f"Saved outputs for {course_code}")


ModuleNotFoundError: No module named 'openai'

In [ ]:
# post counts got smaller, debug cell below:

In [25]:
# debug test to get more data
# === Ivy Debug Cell: Full Filtering Walkthrough ===
"""
Purpose:
Trace end-to-end: raw load → dedupe → literal match → sentiment/help-seeking → per-course save.
Optional: Switch to full-history (all posts) by toggling DAYS_BACK.
"""

import pandas as pd
import sqlite3
from pathlib import Path
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime
import logging

# === Config ===
PROJECT_ROOT = Path("/Users/buddy/Desktop/WGU-Reddit")
DB_PATH = PROJECT_ROOT / "db" / "WGU-Reddit.db"
TOP20_CSV = PROJECT_ROOT / "data" / "output" / "reddit_top_20_mentioned_courses.csv"

DATE_TAG = datetime.now().strftime("%Y-%m-%d")
OUTPUT_DIR = PROJECT_ROOT / "Ivy" / "preprocessed" / DATE_TAG
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

LOG_DIR = PROJECT_ROOT / "Ivy" / "logs"
LOG_DIR.mkdir(parents=True, exist_ok=True)
log_file = LOG_DIR / f"step1_debug_{DATE_TAG}.log"

logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

# === Toggle: days back or ALL ===
DAYS_BACK = 0  # 0 = ALL TIME, 90 = last 90 days

date_filter = f"created_utc >= strftime('%s', 'now', '-{DAYS_BACK} days')" if DAYS_BACK > 0 else "1=1"

# === Load ===
with sqlite3.connect(DB_PATH) as conn:
    df_organic = pd.read_sql_query(
        f"SELECT post_id, title, selftext, created_utc FROM posts WHERE {date_filter}",
        conn
    )
    df_organic['source'] = 'organic'

    df_keyword = pd.read_sql_query(
        f"SELECT post_id, title, selftext, created_utc, search_terms FROM posts_keyword WHERE {date_filter}",
        conn
    )
    df_keyword['source'] = 'keyword'

print(f"Loaded: {len(df_organic)} organic | {len(df_keyword)} keyword")
logging.info(f"Loaded: {len(df_organic)} organic | {len(df_keyword)} keyword")

# === Combine & dedupe ===
df_combined = pd.concat([df_organic, df_keyword], ignore_index=True)
df_combined = df_combined.sort_values('source')  # keep keyword version first
df_combined = df_combined.drop_duplicates(subset=['post_id'], keep='first')
print(f"Combined unique: {len(df_combined)}")
logging.info(f"Combined unique: {len(df_combined)}")

# === Merge text ===
df_combined['Post_Text'] = df_combined.apply(
    lambda r: f"{r['title']}\n{r['selftext']}".strip() if pd.notnull(r['selftext']) else r['title'].strip(),
    axis=1
)

# === Top 20 ===
top20_list = pd.read_csv(TOP20_CSV)['Course Code'].str.upper().unique().tolist()
print(f"Top 20 loaded: {top20_list}")

# === Literal match ===
df_combined['Contains_Literal'] = df_combined['Post_Text'].str.upper().apply(
    lambda t: any(code in t for code in top20_list)
)
literal_hits = df_combined['Contains_Literal'].sum()
print(f"Literal matches: {literal_hits} / {len(df_combined)}")

# === Search terms ===
df_combined['Has_SearchTerms'] = df_combined['search_terms'].notna() & df_combined['search_terms'].str.strip().ne('')
searchterm_hits = df_combined['Has_SearchTerms'].sum()
print(f"Search terms present: {searchterm_hits}")

# === Cross ===
both = df_combined[df_combined['Contains_Literal'] & df_combined['Has_SearchTerms']]
only_literal = df_combined[df_combined['Contains_Literal'] & ~df_combined['Has_SearchTerms']]
only_st = df_combined[~df_combined['Contains_Literal'] & df_combined['Has_SearchTerms']]
print(f"Both literal + search_terms: {len(both)}")
print(f"Only literal: {len(only_literal)}")
print(f"Only search_terms: {len(only_st)}")

# === Filter on literal ===
df_filtered = df_combined[df_combined['Contains_Literal']].copy()

# === VADER ===
analyzer = SentimentIntensityAnalyzer()
df_filtered['VADER'] = df_filtered['Post_Text'].apply(lambda t: analyzer.polarity_scores(t)['compound'])

# === Help-seeking ===
help_phrases = ["need help", "help!", "how do I", "does anyone know"]  # can expand
df_filtered['has_help'] = df_filtered['Post_Text'].str.lower().apply(lambda t: any(p in t for p in help_phrases))

NEG = -0.2
df_final = df_filtered[(df_filtered['VADER'] < NEG) | df_filtered['has_help']].copy()
print(f"After sentiment/help-seeking filter: {len(df_final)}")

# === Save sample ===
for course in top20_list:
    subset = df_final[df_final['Post_Text'].str.upper().str.contains(course)]
    if not subset.empty:
        out_file = OUTPUT_DIR / f"{course}_debug_clean.csv"
        subset.to_csv(out_file, index=False)
        print(f"✅ Saved {course}: {len(subset)} posts")

print(f"✅ Debug log written: {log_file}")

Loaded: 18489 organic | 3952 keyword
Combined unique: 20810
Top 20 loaded: ['C214', 'D427', 'C211', 'C213', 'C207', 'D335', 'D288', 'D287', 'D333', 'D426', 'D315', 'C949', 'D278', 'C215', 'C777', 'D336', 'D197', 'D282', 'D487', 'D316']
Literal matches: 3672 / 20810
Search terms present: 3952
Both literal + search_terms: 3565
Only literal: 107
Only search_terms: 387
After sentiment/help-seeking filter: 799
✅ Saved C214: 68 posts
✅ Saved D427: 88 posts
✅ Saved C211: 60 posts
✅ Saved C213: 45 posts
✅ Saved C207: 70 posts
✅ Saved D335: 66 posts
✅ Saved D288: 35 posts
✅ Saved D287: 28 posts
✅ Saved D333: 58 posts
✅ Saved D426: 66 posts
✅ Saved D315: 32 posts
✅ Saved C949: 36 posts
✅ Saved D278: 27 posts
✅ Saved C215: 30 posts
✅ Saved C777: 66 posts
✅ Saved D336: 22 posts
✅ Saved D197: 32 posts
✅ Saved D282: 30 posts
✅ Saved D487: 22 posts
✅ Saved D316: 27 posts
✅ Debug log written: /Users/buddy/Desktop/WGU-Reddit/Ivy/logs/step1_debug_2025-07-10.log


## New idea: does adding "?" alone, give too much noise, or identify good help-seeking posts?

In [26]:
# === Ivy Debug: Inspect ? vs help_keywords ===
"""
Purpose:
- Detect help-seeking via explicit phrases.
- Detect question mark presence separately.
- Find posts that would ONLY be flagged by ?.
- Print count + sample for manual sanity check.
"""

import pandas as pd

# === Assume df_filtered exists from prior steps ===

# === Baseline help phrases ===


# === Flags ===
df_filtered['Has_Help_Phrase'] = df_filtered['Post_Text'].str.lower().apply(
    lambda t: any(p in t for p in help_keywords)
)

df_filtered['Has_QuestionMark'] = df_filtered['Post_Text'].str.contains(r'\?')

# === Inspect coverage ===
num_help_phrase = df_filtered['Has_Help_Phrase'].sum()
num_qmark = df_filtered['Has_QuestionMark'].sum()

print(f"Posts with help phrase: {num_help_phrase}")
print(f"Posts with question mark: {num_qmark}")

# === Find only-? posts ===
df_only_qmark = df_filtered[
    (df_filtered['Has_QuestionMark']) & (~df_filtered['Has_Help_Phrase'])
]

print(f"Posts flagged ONLY by ?: {len(df_only_qmark)}")

# === Show samples ===
sample = df_only_qmark[['Post_Text']].sample(min(10, len(df_only_qmark)), random_state=42) if not df_only_qmark.empty else None

if sample is not None:
    print("\n=== Example posts caught ONLY by ? ===")
    for idx, row in sample.iterrows():
        print(f"- {row['Post_Text'][:200]}...\n")
else:
    print("No only-? posts to review.")

Posts with help phrase: 858
Posts with question mark: 2395
Posts flagged ONLY by ?: 1761

=== Example posts caught ONLY by ? ===
- Set to start the SWE C# program July 1st with 60% transfer in, realistic to finish in 1 term and tips to do so?
Looking for some tips to prepare myself for when I start, and I guess reality check if i...

- D278 - take the sophia .org equivalent? Or wait and take it at WGU?
I'm planning to start the Data Analytics B.S. at WGU on August 1. Right now, I'm waiting to see what courses might be picked up from...

- Nearing term end date
What happens if I don’t finish a course at the end of my term? I’m currently in the MBA program and it doesn’t look like I will finish C215 in time? Would I just retake the cours...

- Recommended Course Order for 2nd Term
New term starts Aug 1st and my schedule will be a bit busy until November. I have the following classes left, what courses should I register for that are on the l...

- C777 OA entering into the test environmen

In [29]:
from IPython.display import display

# === VADER threshold ===
NEG = -0.2

# === Help phrase flags ===
help_keywords = [
    "need help", "help!", "help with", "any advice", "looking for advice",
    "advice on", "tips on", "looking for tips", "need suggestions",
    "need recommendations", "how do i", "where do i", "where can i",
    "what do i", "when should i", "which should i", "does anyone know",
    "does anyone have", "anyone know how", "can someone help",
    "can anyone help", "stuck on", "struggling with", "cannot figure out",
    "can’t figure out", "having trouble with", "confused about",
    "lost on", "don’t understand", "not sure how", "no idea how",
    "trying to figure out", "help me understand", "explain how",
    "can someone explain", "make sense of", "anyone dealt with",
    "how did you handle", "how did you manage", "what worked for you",
    "am i missing something", "doing something wrong", "what am i doing wrong",
    "should i be", "am i supposed to", "can anyone explain",
    "what's the best way to", "any pointers on"
]

df_filtered['Has_Help_Phrase'] = df_filtered['Post_Text'].str.lower().apply(
    lambda t: any(p in t for p in help_keywords)
)

df_filtered['Has_QuestionMark'] = df_filtered['Post_Text'].str.contains(r'\?')

df_filtered['Fails_VADER'] = df_filtered['VADER'] >= NEG

# === Find posts rescued ONLY by ? ===
df_only_qmark = df_filtered[
    df_filtered['Fails_VADER'] &
    (~df_filtered['Has_Help_Phrase']) &
    (df_filtered['Has_QuestionMark'])
]

print(f"Posts rescued ONLY by ?: {len(df_only_qmark)}")

# === Show sample with full text ===
sample_size = min(50, len(df_only_qmark))

# Expand text display for long posts
pd.set_option('display.max_colwidth', None)

sample_df = df_only_qmark[['VADER', 'Post_Text']].sample(
    sample_size, random_state=42
).reset_index(drop=True)

print(f"Displaying {sample_size} posts that fail VADER, no phrase, but have ?")
display(sample_df)

Posts rescued ONLY by ?: 1412
Displaying 50 posts that fail VADER, no phrase, but have ?


,VADER,Post_Text
0,0.8519,Class Credit Turnaround for Completed Certification\nJust wrapped up D336 earlier this week (Tuesday) with a successful cert exam. How long does it typically take for the class to officially show as finished in WGU? Thanks
1,0.0000,D316 and D317\nAre these two courses (IT Foundations and IT Applications). Both A+ exams for the test outs?
2,0.2340,"C207 new course Aug 1st\nSo I am about to start C207, my last class before the capstone. \n\nI received a pop up that the class will be redesigned effective Aug 1st, so I imagine any progress I make between now and then will be lost, if I don’t finish. \n\nI was planning to try and power through it that quickly, but have heard some horror stories about the class and how difficult it can be. Has anyone heard if the redesign will be better? Should I wait? Is 2 weeks a reasonable amount of time to complete C207 as is? \n\nThank you fellow night owls!"
3,0.9428,"Memorization heavy courses?\nHey all, \n\nI'm looking for opinions on what everyone thinks were memorization heavy courses. For me D315 Network and Security Foundations was ROUGH because it was just all memorizing. I realize this will differ from person to person based on what knowledge they might already have. Feel free to post whatever you feel like regarding if the course required a ton of memorization but I am also specifically interested in opinion regarding my remaining classes which are:\n\nUSER INTERFACE DESIGN – D279\n\nJAVASCRIPT PROGREAMMING – D280\n\nBUSINESS OF IT – PROJECT MANGEMENT – D324\n\nBUSINESS OF IT – APPLICATIONS – D336\n\nHARDWARE AND OPERATIONS SYSTEMS ESSENTIALS – D386\n\nSOFTWARE ENGINEERING – D284\n\nJAVA FRAMEWORKS – D287\n\nJAVA FUNDAMENTALS – D286\n\nIT LEADERSHIP FOUNDATIONS – D370\n\nBACK END PROGRAMMING – D288\n\nADVANCE JAVA – D387\n\nSOFTWARE DESIGN AND QA – D480\n\nDATA STRUCTURES AND ALGOS 1 – C949\n\nMOBILE APPLICATION DVLPMNT (ANDOIRD) – D308\n\nUSER EXPERIENCE DESIGN – D479\n\nCLOUD FOUNDATIONS – D282\n\nSOFTWARE SECURITY AND TESTING – D385\n\nVERSION CONTROL – D197\n\nADVANCED DATA MGMT – D326\n\nSOFTWARE ENGINEERING CAPSTONE – D424"
4,0.3612,"D288 - Tables not populating\nI have this project basically finished, I just can’t get the cart, cart_items, and excursion_cart item tables populating in mySQL. I’ve gone as far as to rewrite my entities, but no dice. Could anyone lend me a hand?"
5,0.7364,"D281 Linux Foundation\nJust scored 800/800 on the exam. To be honest, this was nothing compared to the C777 OA (first course I took).\n\nYou need a 500/800. That is a 62.5% to pass, so try not to stress about it too much. You can do it! Keep pushing!\n\nResources: \nhttps://www.netacad.com/courses/linux-essentials?courseLang=en-US\n\nhttps://wgu.udemy.com/course/linux-essentials-010/learn/lecture/16312160#overview\n\nhttps://noam-alum.github.io/lpi_010_160_exam/"
6,0.9904,"C954 - Information Technology Management PASSED [80% 1.5 Wks]\n&#x200B;\n\nhttps://preview.redd.it/hvw3i2qphh9b1.png?width=765&format=png&auto=webp&s=1056aa8659d84f499c2cb150f68a973174cc8d91\n\nBefore taking this class I did extensive reading and scrolling through reviews and course chatter to see nothing but negativity and people saying it is THE hardest and LEAST structured class in the entire degree plan for MSITM. \n\nI want to make this post to say, if you are basing your study off of people saying things like this, **stop.**\n\nIt has made me way more stressed out than necessary too many times now with people over-complaining courses because they want to accelerate faster and not put any work in.\n\n(Ex: The amount of complaining about C777 - Web Development Applications , I overstressed that course but got a 90% score on my OA first try after a week, if you want my write up on that comment and I'll link it.)\n\n&#x200B;\n\n**I passed with 1.5 Weeks of studying while working a full time job, here is how:**\n\n\-Download the ANKI Cards from Course Chatter, I used like 35% of them for the bigger to